# All Models on Subsets Dataset
 - We suspect that our modified dataset will yield better results than the 'raw' normalized dataset. In this notebook we will investigate this

In [ ]:
#required imports and dependencies

import pandas as pd
import numpy as np 
import warnings 
warnings.filterwarnings("ignore")
from sklearn import preprocessing
from keras import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM, Flatten
from keras.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



#our own confusion matrix module that we developed
import cmatrix as cm

#load data into data frames
xlsPath = r'C:\Users\Daniel Patrick\Documents\uniwork\ada\ada2\Subsets.xlsx'
data = pd.read_excel(xlsPath)

#split into data and class variable
X = data.drop(['date','price increase tomorrow?'], axis =1)
class_var = data['price increase tomorrow?']


#perform normalization on data 
min_max_scaler = preprocessing.MinMaxScaler()
for column in X:
    X[column] = min_max_scaler.fit_transform(X[column].values.reshape(-1,1))

import pywt
import statistics
for column in X:
    coeff = pywt.wavedec(X[column], "haar", level=10)
    sigma = statistics.median(coeff[-1])/0.6745
    threshold = sigma*np.sqrt(2*np.log(len(X[column])))
    coeff[1:] = (pywt.threshold(i, value=threshold) for i in coeff[1:])
    X[column] = pywt.waverec(coeff, "haar")

#select dimensions to reduce to 
inputDims = 3
attributes = SelectKBest(chi2, k=inputDims).fit_transform(X,class_var)


#Create a train test split
split_number = round(len(data)*0.97)
train_attributes,train_class_var,test_attributes,test_class_var = \
attributes[:split_number],class_var[:split_number], \
attributes[split_number:],class_var[split_number:]



In [ ]:
def sequentialNN(train_attributes,train_class_var,test_attributes,test_class_var):
    
    model = Sequential()
    # The imput dim here is the number of cols in the df getting fed into the model
    model.add(Dense(64, input_dim=inputDims, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])


    model.summary()

    model.fit(train_attributes, train_class_var,
             epochs=250, verbose=0, batch_size=128)

    #the predict_classes method returns the binary variable we're looking for
    # for some reason it doesn't output the correct array type, made a 2D array
    pred = model.predict_classes(test_attributes)

    #the flatten method solves this and squashes to a 1D array for evaluation
    predictions = pred.flatten()

    #be sure to import our cmatrix module
    cmat = cm.cmatrix(test_class_var, predictions)
    
    model.reset_states()
    return cmat




In [ ]:
def logRegression(train_attributes,train_class_var,test_attributes,
                  test_class_var):
    
    #Below is the list of hyper-parameters which produced
    #the best results
    logreg = LogisticRegression(penalty='l2', dual=False, tol=0.0001, 
                C=1.0, fit_intercept=True, solver='liblinear')
    logreg.fit(train_attributes, train_class_var)
    predictions = logreg.predict(test_attributes)
    
    cmat = cm.cmatrix(test_class_var, predictions)
    return cmat

In [ ]:
def randomForest(train_attributes,train_class_var,test_attributes,test_class_var):
    
    random_forest = RandomForestClassifier(n_estimators=20)
    random_forest.fit(train_attributes, train_class_var)
    predictions = random_forest.predict(test_attributes)
    
    cmat = cm.cmatrix(test_class_var, predictions)
    return cmat

In [ ]:
def svc(train_attributes,train_class_var,test_attributes,test_class_var):
    
    svc = SVC()
    svc.fit(train_attributes, train_class_var)
    predictions = svc.predict(test_attributes)
    
    cmat = cm.cmatrix(test_class_var, predictions)
    return cmat
    

In [ ]:
def perceptron(train_attributes,train_class_var,test_attributes,test_class_var):
    
    perceptron = Perceptron()
    perceptron.fit(train_attributes, train_class_var)
    predictions = perceptron.predict(test_attributes)
    
    cmat = cm.cmatrix(test_class_var, predictions)
    return cmat

In [ ]:
sequentialNN(train_attributes,train_class_var,test_attributes,test_class_var)

In [ ]:
logRegression(train_attributes,train_class_var,test_attributes,test_class_var)

In [ ]:
perceptron(train_attributes,train_class_var,test_attributes,test_class_var)

In [ ]:
randomForest(train_attributes,train_class_var,test_attributes,test_class_var)

In [ ]:
svc(train_attributes,train_class_var,test_attributes,test_class_var)